## Modified RNN or LSTM model for Sentiment Analysis of  Amazon reviews

In [1]:
import numpy as np
import pandas as pd
import os
import re
import nltk
from nltk.corpus import stopwords
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Activation, Dropout, Dense
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [2]:
os.chdir("C:\\Users\Venkateswara Rao\\Desktop")

In [3]:
Amazon=pd.read_csv("amazon_cells_labelled.txt", delimiter="\t", header=None, names=["review","label"])


In [4]:
Amazon.head()

,review,label
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [5]:
### Data Frame Attributes 
Amazon.ndim

2

In [6]:
Amazon.shape

(1000, 2)

In [7]:
Amazon["label"].value_counts() 

1    500
0    500
Name: label, dtype: int64

In [8]:
import nltk 
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lematizer=WordNetLemmatizer() 

## Preprocessing 

In [67]:
X=[]
for i in range(0,len(Amazon)):
    review=re.sub(r'\s+', ' ', Amazon['review'][i])### Removing multiple spaces
    
    #review=re.sub('\d+"', '', Amazon['review'][i])### Removing Numbers
    review=re.sub('[^a-zA-Z]', ' ', Amazon['review'][i])## # Remove  Numbers and special characters( punctuations)
    #review=re.sub(r'\s+[a-zA-Z]\s+', ' ', Amazon['review'][i] )### Single character removal
    review=review.lower()
    review=review.split()### String Split function act as unigram word tokenization 
    review=[lematizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    #review=' '.join(review)
    X.append(review)

In [68]:
X

[['way', 'plug', 'u', 'unless', 'go', 'converter'],
 ['good', 'case', 'excellent', 'value'],
 ['great', 'jawbone'],
 ['tied', 'charger', 'conversation', 'lasting', 'minutesmajor', 'problem'],
 ['mic', 'great'],
 ['jiggle', 'plug', 'get', 'line', 'right', 'get', 'decent', 'volume'],
 ['several',
  'dozen',
  'several',
  'hundred',
  'contact',
  'imagine',
  'fun',
  'sending',
  'one',
  'one'],
 ['razr', 'owneryou', 'must'],
 ['needle', 'say', 'wasted', 'money'],
 ['waste', 'money', 'time'],
 ['sound', 'quality', 'great'],
 ['impressed', 'going', 'original', 'battery', 'extended', 'battery'],
 ['two',
  'seperated',
  'mere',
  'ft',
  'started',
  'notice',
  'excessive',
  'static',
  'garbled',
  'sound',
  'headset'],
 ['good', 'quality', 'though'],
 ['design', 'odd', 'ear', 'clip', 'comfortable'],
 ['highly', 'recommend', 'one', 'blue', 'tooth', 'phone'],
 ['advise', 'everyone', 'fooled'],
 ['far', 'good'],
 ['work', 'great'],
 ['click',
  'place',
  'way',
  'make',
  'wonder',

In [53]:
X[0:5]

[['way', 'plug', 'u', 'unless', 'go', 'converter'],
 ['good', 'case', 'excellent', 'value'],
 ['great', 'jawbone'],
 ['tied', 'charger', 'conversation', 'lasting', 'minutesmajor', 'problem'],
 ['mic', 'great']]

In [54]:
### With List  join function 
X=[]
for i in range(0,len(Amazon)):
    review=re.sub(r'\s+[a-zA-Z]\s+', ' ', Amazon['review'][i] )### Single character removal
    review=re.sub(r'\s+', ' ', Amazon['review'][i])### Removing multiple spaces
    review=re.sub('\d+"', '', Amazon['review'][i])### Removing Numbers
    review=re.sub('[^a-zA-Z]', ' ', Amazon['review'][i])## # Remove  Numbers and special characters( punctuations) 
    review=review.lower()
    review=review.split()
    review=[lematizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    X.append(review)

In [55]:
X

['way plug u unless go converter',
 'good case excellent value',
 'great jawbone',
 'tied charger conversation lasting minutesmajor problem',
 'mic great',
 'jiggle plug get line right get decent volume',
 'several dozen several hundred contact imagine fun sending one one',
 'razr owneryou must',
 'needle say wasted money',
 'waste money time',
 'sound quality great',
 'impressed going original battery extended battery',
 'two seperated mere ft started notice excessive static garbled sound headset',
 'good quality though',
 'design odd ear clip comfortable',
 'highly recommend one blue tooth phone',
 'advise everyone fooled',
 'far good',
 'work great',
 'click place way make wonder long mechanism would last',
 'went motorolas website followed direction could get pair',
 'bought use kindle fire absolutely loved',
 'commercial misleading',
 'yet run new battery two bar thats three day without charging',
 'bought mother problem battery',
 'great pocket pc phone combination',
 'ive owned 

In [56]:
X[0:5]

['way plug u unless go converter',
 'good case excellent value',
 'great jawbone',
 'tied charger conversation lasting minutesmajor problem',
 'mic great']

## Second way of preprocessing

In [14]:
from nltk.corpus import stopwords


In [15]:
stops = stopwords.words('english')

In [16]:
from nltk.stem import WordNetLemmatizer

In [17]:
lemm=WordNetLemmatizer()

In [46]:
def _remove_noise(input_text):
    #input_text = str(input_text).encode('ascii', 'ignore')## Now 'b ' not came in the begining of sentences
    # there are some when we are encoding  entire text to ascii , only special charecters are predefined
    # like japanese  and other languages it can not convert to ascii ,there are non as 
    input_text = str(input_text).replace("<br />", " ")
    input_text = str(input_text).replace(",", "")
    input_text = str(input_text).replace("\'" ,"")
    input_text = str(input_text).replace("\\", "")
    input_text = str(input_text).replace("-", "")
    input_text = str(input_text).replace(".", "")
    #input_text  =re.sub(r'\s+[a-zA-Z]\s+', ' ',input_text )### Single character removal
    input_text  =re.sub(r'\s+', ' ',input_text )### Removing multiple spaces
     
    #input_text = re.sub("\d+","",input_text)
    input_text = re.sub("[^a-zA-Z]"," ",input_text)
    words      =str(input_text).split()
    noise_free_words =  [word for word in words if word.lower() not in stops]
    lower_words =[word.lower() for word in noise_free_words]
    lower_words=[lemm.lemmatize(w) for w in lower_words]
                                         
    return lower_words   

In [69]:
_remove_noise(Amazon['review'])

['way',
 'plug',
 'u',
 'unless',
 'go',
 'converter',
 'good',
 'case',
 'excellent',
 'value',
 'great',
 'jawbone',
 'tied',
 'charger',
 'conversation',
 'lasting',
 'minutesmajor',
 'mic',
 'great',
 'screen',
 'get',
 'smudged',
 'easily',
 'touch',
 'ear',
 'face',
 'piece',
 'junk',
 'lose',
 'call',
 'phone',
 'item',
 'match',
 'picture',
 'thing',
 'disappoint',
 'infra',
 'red',
 'port',
 'irda',
 'answer',
 'call',
 'unit',
 'never',
 'worked',
 'name',
 'review',
 'length',
 'dtype',
 'object']

In [41]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Venkateswara Rao\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [42]:
Amazon["cleaned"] = Amazon.review.apply(_remove_noise)
Amazon.head()


,review,label,cleaned
0,way plug u unless go converter,0,"[way, plug, u, unless, go, converter]"
1,good case excellent value,1,"[good, case, excellent, value]"
2,great jawbone,1,"[great, jawbone]"
3,tied charger conversation lasting minutesmajor...,0,"[tied, charger, conversation, lasting, minutes..."
4,mic great,1,"[mic, great]"


In [23]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Venkateswara
[nltk_data]     Rao\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [24]:
#from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
Amazon['review'] = Amazon.cleaned.map(lambda x: ' '.join([lemmatizer.lemmatize(y) for y in x]))
Amazon.review.head()

0                       way plug u unless go converter
1                            good case excellent value
2                                        great jawbone
3    tied charger conversation lasting minutesmajor...
4                                            mic great
Name: review, dtype: object

In [25]:
X=Amazon['review']

## Preparing target variable and common for above 2 types of preprocessing

In [26]:
y=Amazon['label']

In [27]:
### Spliting Train and test datasets
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=123)

## Preparing the Embedding Layer

In [28]:
tokenizer = Tokenizer(num_words=1500)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [29]:
## truncate and pad input sequences.
# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

## Create Modified RNN or  LSTM model

In [30]:
# create the model
embedding_vector_length = 32
model = Sequential()

In [31]:
model.add(Embedding(vocab_size, embedding_vector_length,\
                       input_length=maxlen)) # Embedding is euivalent to one hot encoding with lesser dimentionality



In [32]:
model.add(LSTM(100))

In [33]:
model.add(Dense(1, activation='sigmoid'))


In [34]:
model.compile(loss='binary_crossentropy',
                 optimizer='adam',
                   metrics=['accuracy'])

In [35]:
model.fit(X_train, y_train,
     validation_data=(X_test, y_test),
           epochs=3, batch_size=64)


C:\Users\Venkateswara Rao\Anaconda3\envs\venkat\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 800 samples, validate on 200 samples
Epoch 1/3
800/800 [==============================] - 4s 5ms/step - loss: 0.6937 - accuracy: 0.4863 - val_loss: 0.6932 - val_accuracy: 0.4950
Epoch 2/3
800/800 [==============================] - 3s 4ms/step - loss: 0.6934 - accuracy: 0.4837 - val_loss: 0.6931 - val_accuracy: 0.5050
Epoch 3/3
800/800 [==============================] - 3s 4ms/step - loss: 0.6937 - accuracy: 0.4913 - val_loss: 0.6934 - val_accuracy: 0.4950


In [36]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)


In [37]:
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 49.50%
